In [1]:
from PIL import Image
import os
import numpy as np
import tensorflow as tf

In [2]:
# Hàm tính khoảng cách Euclidean giữa hai vector
def euclidean_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

In [3]:
# Hàm tính vector đặc trưng khuôn mặt bằng mô hình FaceNet
def extract_face_features(image_path, model):
    image = Image.open(image_path)
    image = image.convert('RGB')

    # Tiền xử lý ảnh
    processed_image = preprocess_image(image)

    # Trích xuất đặc trưng khuôn mặt
    features = model.predict(np.expand_dims(processed_image, axis=0))[0]

    return features

In [4]:
# Hàm tiền xử lý ảnh
def preprocess_image(image):
    # Chuyển đổi ảnh thành numpy array
    image = np.array(image)

    # Chuẩn hóa giá trị pixel trong khoảng [-1, 1]
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)

    # Resize ảnh về kích thước (224, 224) (phù hợp với mô hình FaceNet)
    image = tf.image.resize(image, (224, 224))

    return image


In [5]:
# Tạo mô hình FaceNet đơn giản
def create_facenet_model():
    model = tf.keras.Sequential()

    # Lớp tích chập 1
    model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))

    # Lớp tích chập 2
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))

    # Lớp tích chập 3
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))

    # Lớp Flatten
    model.add(tf.keras.layers.Flatten())

    # Lớp fully connected 1
    model.add(tf.keras.layers.Dense(128, activation='relu'))

    # Lớp fully connected 2 (đầu ra là vector đặc trưng 128 chiều)
    model.add(tf.keras.layers.Dense(128))

    return model

In [6]:
# Đường dẫn đến thư mục gốc chứa các folder của các người
data_dir = "/content/Picture"


In [7]:
# Danh sách ảnh khuôn mặt
X_train = []
# Danh sách nhãn tương ứng
Y_train = []

In [8]:
# Duyệt qua các folder của các người
for person_folder in os.listdir(data_dir):
    person_dir = os.path.join(data_dir, person_folder)
    # Kiểm tra xem đối tượng có phải là folder hay không
    if os.path.isdir(person_dir):
        # Duyệt qua các ảnh khuôn mặt của từng người
        for filename in os.listdir(person_dir):
            image_path = os.path.join(person_dir, filename)
            # Đọc ảnh khuôn mặt sử dụng PIL
            image = Image.open(image_path)
            image = image.convert('RGB')
            # Tiền xử lý ảnh (nếu cần)
            processed_image = preprocess_image(image)
            # Thêm ảnh và nhãn vào danh sách
            X_train.append(processed_image)
            Y_train.append(person_folder)


In [9]:
# Số lượng người trong tập huấn luyện
num_classes = len(set(Y_train))

In [10]:
# Chuyển đổi danh sách ảnh và nhãn thành numpy array
X_train = np.array(X_train)
Y_train = np.array(Y_train)


In [11]:
# Xây dựng mô hình FaceNet
model = create_facenet_model()

In [12]:
# Compile mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
# Huấn luyện mô hình
# Tạo từ điển ánh xạ nhãn chuỗi thành số nguyên
label_to_int = {label: i for i, label in enumerate(set(Y_train))}

In [14]:
# Chuyển đổi danh sách nhãn thành các số nguyên tương ứng
Y_train_int = np.array([label_to_int[label] for label in Y_train])

In [15]:
# Huấn luyện mô hình
model.fit(X_train, Y_train_int, epochs=10, batch_size=32)

Epoch 1/10
31/31 [==============================] - 70s 2s/step - loss: 11.4874 - accuracy: 0.1473
Epoch 2/10
31/31 [==============================] - 63s 2s/step - loss: 4.8498 - accuracy: 0.0820
Epoch 3/10
31/31 [==============================] - 62s 2s/step - loss: 4.8520 - accuracy: 0.0000e+00
Epoch 4/10
31/31 [==============================] - 58s 2s/step - loss: 4.8520 - accuracy: 0.0000e+00
Epoch 5/10
31/31 [==============================] - 58s 2s/step - loss: 4.8520 - accuracy: 0.0000e+00
Epoch 6/10
31/31 [==============================] - 56s 2s/step - loss: 4.8520 - accuracy: 0.0000e+00
Epoch 7/10
31/31 [==============================] - 57s 2s/step - loss: 4.8520 - accuracy: 0.0000e+00
Epoch 8/10
31/31 [==============================] - 59s 2s/step - loss: 4.8520 - accuracy: 0.0000e+00
Epoch 9/10
31/31 [==============================] - 57s 2s/step - loss: 4.8520 - accuracy: 0.0000e+00
Epoch 10/10
31/31 [==============================] - 55s 2s/step - loss: 4.8520 - accurac

In [17]:

# # Đường dẫn đến thư mục chứa ảnh
# image_folder = "/content/Picture/person1"

# # Danh sách đặc trưng của các ảnh
# all_face_features = []

# # Duyệt qua từng ảnh trong thư mục
# for filename in os.listdir(image_folder):
#     image_path = os.path.join(image_folder, filename)
#     # Trích xuất đặc trưng từ ảnh và thêm vào danh sách
#     face_features = extract_face_features(image_path, model)
#     all_face_features.append(face_features)

# # Danh sách đặc trưng của tất cả các ảnh trong thư mục
# print(all_face_features)


# Danh sách đường dẫn thư mục chứa ảnh khuôn mặt
image_folders = ["/content/Picture/Ariel_Sharon", "/content/Picture/Colin_Powell", "/content/Picture/Donald_Rumsfeld", "/content/Picture/George_W_Bush"]

# Danh sách đặc trưng của các ảnh
all_face_features = []

# Duyệt qua từng thư mục ảnh
for folder_path in image_folders:
    # Duyệt qua từng ảnh trong thư mục
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        # Trích xuất đặc trưng từ ảnh và thêm vào danh sách
        face_features = extract_face_features(image_path, model)
        all_face_features.append(face_features)

# Danh sách đặc trưng của tất cả các ảnh trong các thư mục
print(all_face_features)


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
       -0.01384276, -0.01014201, -0.00998071, -0.01970787, -0.01644024,
       -0.01228203, -0.00370066, -0.01621357, -0.01883198, -0.02217689,
       -0.01691473, -0.00496854, -0.00701799, -0.01609367, -0.01345269,
       -0.02464259, -0.02767619, -0.01654867, -0.01591909, -0.01015327,
       -0.02564114, -0.01792433, -0.01797746, -0.01499418, -0.02593853,
       -0.01073865, -0.01587361, -0.01717536, -0.00689112, -0.01583303,
       -0.01366123, -0.01969935, -0.01783553, -0.01939919, -0.01183688,
       -0.01389038, -0.01203755, -0.0250619 , -0.01913886, -0.01547609,
       -0.00811741, -0.02424836, -0.01465515, -0.01216562, -0.00791809,
       -0.01118463, -0.00874582, -0.00871904, -0.01227458, -0.01657725,
       -0.02074121, -0.01095309, -0.01205211, -0.02207935, -0.01723476,
       -0.00642315, -0.01034363, -0.01185603, -0.01643247, -0.00865505,
       -0.01336016, -0.02288081, -0.01828953, -0.01752655, -0.01408667,
       

In [29]:
# Đường dẫn đến ảnh khuôn mặt mới cần nhận diện
new_image_path = "/content/Picture/Ariel_Sharon_0019.jpg"

In [30]:

# Trích xuất đặc trưng của ảnh mới
new_face_features = extract_face_features(new_image_path, model)

1/1 [==============================] - 0s 45ms/step


In [31]:
# Danh sách khoảng cách giữa ảnh mới và các ảnh đã biết trong tập huấn luyện
distances = []
for face_features in all_face_features:
    distance = euclidean_distance(new_face_features, face_features)
    distances.append(distance)

In [32]:
# Xác định ngưỡng để quyết định khuôn mặt mới thuộc vào danh sách người đã biết hay không
threshold = 0.5

In [33]:
# Tìm khoảng cách nhỏ nhất và vị trí tương ứng trong danh sách khoảng cách
min_distance = min(distances)
min_distance_index = distances.index(min_distance)

In [34]:
# Kiểm tra xem khuôn mặt mới có thuộc vào danh sách người đã biết hay không
if min_distance <= threshold:
    recognized_person = Y_train[min_distance_index]
    print("Khuôn mặt mới thuộc về người:", recognized_person)
else:
    print("Khuôn mặt mới không thuộc danh sách người đã biết")

Khuôn mặt mới thuộc về người: Ariel_Sharon
